In [ ]:
import os
import re
import pickle
import pandas as pd
import numpy as np
import seaborn as sn
import keras
from keras.preprocessing import text, sequence
from keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import Model, Sequential
from keras import backend as K
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='loss', patience=10)#, verbose=1)
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
lb_enc = LabelEncoder()
import nltk
import string 

In [ ]:
data_train = pd.read_excel('/kaggle/input/spam-review-detection/data_train.xlsx')
data_dev = pd.read_excel('/kaggle/input/spam-review-detection/data_dev.xlsx')
data_test = pd.read_excel('/kaggle/input/spam-review-detection/data_test.xlsx')

In [ ]:
y_train = lb_enc.fit_transform(data_train['Label'])

y_dev = lb_enc.fit_transform(data_dev['Label'])

y_test = lb_enc.fit_transform(data_test['Label'])

In [ ]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(data_train['transformed_text'])
tokenizer.fit_on_texts(data_dev['transformed_text'])
tokenizer.fit_on_texts(data_test['transformed_text'])

text_to_sequence_train = tokenizer.texts_to_sequences(data_train['transformed_text']) 
text_to_sequence_dev = tokenizer.texts_to_sequences(data_dev['transformed_text']) 
text_to_sequence_test = tokenizer.texts_to_sequences(data_test['transformed_text']) 

In [ ]:
max_length_sequence_train = max([len(i) for i in text_to_sequence_train])
 
padded_train = pad_sequences(text_to_sequence_train, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
max_length_sequence_train
padded_train

In [ ]:
padded_dev = pad_sequences(text_to_sequence_dev, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
len(padded_dev)

In [ ]:
padded_test = pad_sequences(text_to_sequence_test, maxlen=max_length_sequence_train, 
                                    padding = "pre") 
len(padded_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, SimpleRNN, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
len(tokenizer.word_index)+1

In [ ]:
VOC_SIZE = len(tokenizer.word_index)+1 # 18359
def create_model():
    
      model = Sequential()
      model.add(Embedding(VOC_SIZE, 300, input_length=max_length_sequence_train))
      model.add(Dropout(0.8))
      model.add(GRU(400, activation = 'tanh', recurrent_activation = 'sigmoid', return_sequences = True))
      model.add(Dropout(0.86))
      model.add(GRU(400))
      model.add(Dropout(0.5))
#       model.add(Dense(256, activation="relu"))
#       model.add(Dropout(0.5))
      model.add(Dense(1, activation = "sigmoid", name='Classification'))
      return model
gru_model = create_model()
gru_model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
gru_model.summary()

In [ ]:
history = gru_model.fit(padded_train, 
                        y_train, epochs=40, batch_size=64, 
                        validation_data=(padded_dev, y_dev), callbacks = [early_stop])

In [ ]:
y_pred_train = gru_model.predict(padded_train)
y_pred_classes_train = np.round(y_pred_train)

# Evaluate performance on test set
acc = accuracy_score(y_train, y_pred_classes_train)
prec = precision_score(y_train, y_pred_classes_train)
f1_macro = f1_score(y_train, y_pred_classes_train, average='macro')
cm = confusion_matrix(y_train, y_pred_classes_train)
print("Accuracy: ", acc)
print("Precision: ", prec)
print("F1 Macro: ", f1_macro)
print("Confusion Matrix: \n", cm)

In [ ]:
y_pred_dev = gru_model.predict(padded_dev)
y_pred_classes_dev = np.round(y_pred_dev)

# Evaluate performance on test set
acc = accuracy_score(y_dev, y_pred_classes_dev)
prec = precision_score(y_dev, y_pred_classes_dev)
f1_macro = f1_score(y_dev, y_pred_classes_dev, average='macro')
cm = confusion_matrix(y_dev, y_pred_classes_dev)
print("Accuracy: ", acc)
print("Precision: ", prec)
print("F1 Macro: ", f1_macro)
print("Confusion Matrix: \n", cm)

In [ ]:
y_pred = gru_model.predict(padded_test)
y_pred_classes = np.round(y_pred)

# Evaluate performance on test set
acc = accuracy_score(y_test, y_pred_classes)
prec = precision_score(y_test, y_pred_classes)
f1_macro = f1_score(y_test, y_pred_classes_test, average='macro')
cm = confusion_matrix(y_test, y_pred_classes)
print("Accuracy: ", acc)
print("Precision: ", prec)
print("F1 Macro: ", f1_macro)
print("Confusion Matrix: \n", cm)